<a href="https://colab.research.google.com/github/Capstone-Project-CCNY/AI-Driven-Mental-Disorder-Detection-/blob/main/DevikaRAF_DB_EfficientNetB0_Epoch10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Step 1: Upload Kaggle API Key & Install Kaggle

In [1]:
# Upload Kaggle API Key (Upload kaggle.json)
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dsalimkumar","key":"05282af694ebf4c57e46ee9f811b2b31"}'}

In [2]:
# Install Kaggle API
!pip install kaggle

# Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

 Step 2: Download & Extract RAF-DB Dataset

In [3]:
# Download RAF-DB dataset from Kaggle (Replace with actual dataset ID if available)
!kaggle datasets download -d shuvoalok/raf-db-dataset


Dataset URL: https://www.kaggle.com/datasets/shuvoalok/raf-db-dataset
License(s): other
 53% 20.0M/37.7M [00:00<00:00, 102MB/s] 
100% 37.7M/37.7M [00:00<00:00, 146MB/s]


In [4]:
# Extract the dataset
import zipfile

with zipfile.ZipFile("raf-db-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("raf-db")

import os

# Check extracted files
data_path = "raf-db"
print("Extracted files:", os.listdir(data_path))

Extracted files: ['test_labels.csv', 'train_labels.csv', 'DATASET']


 Step 3: Reinstall Specific TensorFlow Version

In [5]:
# Reinstall specific TensorFlow version (if needed)
!pip install tensorflow==2.15.0 tensorflow-addons==0.23.0



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully u

In [ ]:
# Restart runtime to apply changes
import os
os._exit(00)  # Restart runtime

In [1]:
import tensorflow as tf

# Print TensorFlow version
print("TensorFlow Version:", tf.__version__)

# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))
import tensorflow as tf

TensorFlow Version: 2.15.0
GPU Available: []


 Step 4: Optimize TensorFlow for GPU

In [2]:
# Optimize GPU Memory Usage
import os
import tensorflow as tf
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow debug logs
tf.get_logger().setLevel('ERROR')

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print("GPU Optimization Complete!")


GPU Optimization Complete!


Step 5: Load & Preprocess RAF-DB Dataset

In [3]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set Image Properties
IMG_SIZE = 224
BATCH_SIZE = 32

# Define Data Augmentation
train_datagen = ImageDataGenerator(
     rescale=1./255,         # Normalize pixel values
    rotation_range=10,      # Rotate images randomly
    width_shift_range=0.1,  # Shift images horizontally
    height_shift_range=0.1, # Shift images vertically
    horizontal_flip=True    # Flip images horizontally
)

val_datagen = ImageDataGenerator(rescale=1./255) # Only normalize validation images

# Load Images from Directories
train_ds = train_datagen.flow_from_directory(
    "raf-db/DATASET/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_ds = val_datagen.flow_from_directory(
    "raf-db/DATASET/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

print("Dataset Loaded Successfully!")

Found 12271 images belonging to 7 classes.
Found 3068 images belonging to 7 classes.
Dataset Loaded Successfully!


 Step 6: Load EfficientNetB0 & Build Model

In [4]:
# Import Model Components
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load Pretrained EfficientNetB0 (Feature Extractor)
base_model =                                      (weights="imagenet", include_top=False)
base_model.trainable = False  # Keep pretrained weights frozen

# Build Model
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
outputs = Dense(7, activation="softmax")(x)  # 7 Emotion Classes for RAF-DB

# Compile Model
model = Model(inputs, outputs)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Display Model Summary
model.summary()


16705208/16705208 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                              

 Step 7: Setup Checkpoints (Every 5 Epochs)

In [5]:
# Define Checkpoint Callback
checkpoint_path = "rafdb_checkpoints/weights_epoch_{epoch:02d}.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=5 * (len(train_ds)),  # Save every 5 epochs
    verbose=1
)


 Step 8: Train Model with Checkpoints

In [ ]:
EPOCHS = 50  # Adjust as needed

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback],  # Save checkpoints every 5 epochs
    verbose=1
)

print("Training Completed!")

# Save Trained Model
model.save("efficientnetb0_rafdb_final.keras")
print("Model Saved Successfully!")

Epoch 1/50
384/384 [==============================] - 1022s 3s/step - loss: 1.8670 - accuracy: 0.2785 - val_loss: 1.6688 - val_accuracy: 0.3862
Epoch 2/50
384/384 [==============================] - 1004s 3s/step - loss: 1.6568 - accuracy: 0.3881 - val_loss: 1.6328 - val_accuracy: 0.3862
Epoch 3/50
384/384 [==============================] - 1001s 3s/step - loss: 1.6446 - accuracy: 0.3889 - val_loss: 1.6315 - val_accuracy: 0.3862
Epoch 4/50
384/384 [==============================] - 983s 3s/step - loss: 1.6419 - accuracy: 0.3889 - val_loss: 1.6307 - val_accuracy: 0.3862
Epoch 5/50
383/384 [============================>.] - ETA: 2s - loss: 1.6404 - accuracy: 0.3890
Epoch 5: saving model to rafdb_checkpoints/weights_epoch_05.h5
384/384 [==============================] - 995s 3s/step - loss: 1.6409 - accuracy: 0.3889 - val_loss: 1.6306 - val_accuracy: 0.3862
Epoch 6/50
384/384 [==============================] - 896s 2s/step - loss: 1.6409 - accuracy: 0.3889 - val_loss: 1.6317 - val_accuracy